<a href="https://colab.research.google.com/github/ayush139/colllab/blob/main/NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np 
import pandas as pd 

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re

In [2]:
from google.colab import files
  
  
uploaded = files.upload()

Saving tr_dt_manual.xlsx to tr_dt_manual.xlsx


In [3]:

import pandas as pd
import io
  
df = pd.read_excel(io.BytesIO(uploaded['tr_dt_manual.xlsx']))
print(df)

       index  ...     Label
0        0.0  ...  negative
1        1.0  ...  positive
2        2.0  ...  negative
3        3.0  ...   neutral
4        4.0  ...   neutral
...      ...  ...       ...
10747    NaN  ...  negative
10748    NaN  ...  negative
10749    NaN  ...  negative
10750    NaN  ...  negative
10751    NaN  ...  negative

[10752 rows x 6 columns]


In [5]:
data = df[['Corpus','Label']]
print(data)

                                                  Corpus     Label
0                       billing time take extremely long  negative
1      staff try her best to stand along to process t...  positive
2      however waste of time for involve during such ...  negative
3                      give time space more variety need   neutral
4      just want to recommend that you can think to g...   neutral
...                                                  ...       ...
10747                        billing time extremely high  negative
10748  but when try to utilize the same for purchase ...  negative
10749               make charge too high compare to gold  negative
10750                               labour cost too high  negative
10751            price more high compare to market price  negative

[10752 rows x 2 columns]


In [6]:
for idx,row in data.iterrows():
    row[0] = row[0].replace('rt',' ')
    
max_fatures = 5000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['Corpus'].values)
X = tokenizer.texts_to_sequences(data['Corpus'].values)
X = pad_sequences(X)

embed_dim = 542
lstm_out = 196

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(3,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 542, 542)          2710000   
_________________________________________________________________
spatial_dropout1d (SpatialDr (None, 542, 542)          0         
_________________________________________________________________
lstm (LSTM)                  (None, 196)               579376    
_________________________________________________________________
dense (Dense)                (None, 3)                 591       
Total params: 3,289,967
Trainable params: 3,289,967
Non-trainable params: 0
_________________________________________________________________
None


In [7]:

Y = pd.get_dummies(data['Label']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)


(7203, 542) (7203, 3)
(3549, 542) (3549, 3)


In [8]:
batch_size = 32
model.fit(X_train, Y_train, epochs = 7, batch_size=batch_size)

Epoch 1/7
226/226 [==============================] - 1244s 5s/step - loss: 0.7859 - accuracy: 0.6352
Epoch 2/7
226/226 [==============================] - 1247s 6s/step - loss: 0.4252 - accuracy: 0.8381
Epoch 3/7
226/226 [==============================] - 1255s 6s/step - loss: 0.3299 - accuracy: 0.8807
Epoch 4/7
226/226 [==============================] - 1240s 5s/step - loss: 0.2823 - accuracy: 0.8960
Epoch 5/7
226/226 [==============================] - 1236s 5s/step - loss: 0.2397 - accuracy: 0.9118
Epoch 6/7
226/226 [==============================] - 1230s 5s/step - loss: 0.2313 - accuracy: 0.9158
Epoch 7/7
226/226 [==============================] - 1240s 5s/step - loss: 0.1810 - accuracy: 0.9374


In [20]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [21]:
# create on Colab directory
model.save('model.h5')    
model_file = drive.CreateFile({'title' : 'model.h5'})
model_file.SetContentFile('model.h5')
model_file.Upload()

In [22]:

# download to google drive
drive.CreateFile({'id': model_file.get('id')})

GoogleDriveFile({'id': '1xXzok0YZiJCIkSDhTSbrItHf1m8t1-Kw'})

In [76]:
sentiment = model.predict(X_test,batch_size=1)

In [77]:
sentiment

array([[1.4498971e-02, 6.0041088e-01, 3.8509011e-01],
       [2.0616327e-03, 9.7174221e-04, 9.9696666e-01],
       [4.1381560e-02, 8.5893136e-01, 9.9687085e-02],
       ...,
       [2.5329827e-03, 9.8441154e-01, 1.3055458e-02],
       [4.1771257e-01, 4.8539203e-01, 9.6895434e-02],
       [6.7726774e-07, 3.6393347e-05, 9.9996293e-01]], dtype=float32)

In [78]:
pred=pd.DataFrame(sentiment,columns=["Negative","Neutral","Positive"])

In [79]:
pred['Max']=pred.max(axis = 1)

In [80]:
pred.head()

,Negative,Neutral,Positive,Max
0,0.014499,0.600411,0.385090,0.600411
1,0.002062,0.000972,0.996967,0.996967
2,0.041382,0.858931,0.099687,0.858931
3,0.021206,0.957437,0.021357,0.957437
4,0.041564,0.935649,0.022786,0.935649


In [81]:
for i in range(0,len(pred)):
  if (pred['Negative'][i]==pred['Max'][i]):
    pred['Negative'][i]=1
    pred['Neutral'][i]=0
    pred['Positive'][i]=0
  elif (pred['Neutral'][i]==pred['Max'][i]):
    pred['Negative'][i]=0
    pred['Neutral'][i]=1
    pred['Positive'][i]=0
  else:    
    pred['Negative'][i]=0
    pred['Neutral'][i]=0
    pred['Positive'][i]=1

In [84]:
pred.head()

,Negative,Neutral,Positive
0,0.0,1.0,0.0
1,0.0,0.0,1.0
2,0.0,1.0,0.0
3,0.0,1.0,0.0
4,0.0,1.0,0.0


In [83]:
pred.drop(['Max'],axis=1,inplace=True)

In [32]:
ytest=pd.DataFrame(Y_test,columns=["Negative","Neutral","Positive"])

In [75]:
pred.head()

,Negative,Neutral,Positive
0,0.000000,0.000000,0.385090
1,0.002062,0.000972,0.996967
2,0.041382,0.858931,0.099687
3,0.021206,0.957437,0.021357
4,0.041564,0.935649,0.022786


In [34]:
ytest.head()

,Negative,Neutral,Positive
0,0,1,0
1,0,0,1
2,0,1,0
3,0,1,0
4,0,1,0


In [36]:
from google.colab import files
pred.to_csv('pred.csv') 
files.download('pred.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [37]:
from google.colab import files
  
  
uploaded = files.upload()


Saving pred.csv to pred (1).csv


In [41]:
pred = pd.read_csv(io.BytesIO(uploaded['pred.csv']))
print(pred)

      Negative  Neutral  Positive
0            0        1         0
1            0        0         1
2            0        1         0
3            0        1         0
4            0        1         0
...        ...      ...       ...
3544         1        0         0
3545         0        0         1
3546         0        1         0
3547         0        1         0
3548         0        0         1

[3549 rows x 3 columns]


In [85]:
from sklearn.metrics import classification_report
print(classification_report(ytest, pred,target_names=ytest.columns))

              precision    recall  f1-score   support

    Negative       0.76      0.74      0.75       979
     Neutral       0.72      0.75      0.74      1137
    Positive       0.89      0.88      0.88      1433

   micro avg       0.80      0.80      0.80      3549
   macro avg       0.79      0.79      0.79      3549
weighted avg       0.80      0.80      0.80      3549
 samples avg       0.80      0.80      0.80      3549



In [86]:
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

111/111 - 81s - loss: 0.6348 - accuracy: 0.8002
score: 0.63
acc: 0.80
